In [ ]:
# =========================
# RNN OCR (A–Z Dataset) — Keras / TensorFlow
# =========================
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, callbacks, optimizers
import tensorflow as tf
import os

# --- Load or prepare your dataset ---
# Example:
import pandas as pd
data = pd.read_csv("A_Z Handwritten Data.csv").astype('float32')
X = data.drop('0', axis=1).values.reshape(-1,28,28,1) / 255.0
y_labels = data['0'].values
y = tf.keras.utils.to_categorical(y_labels, num_classes=26)

# Ensure arrays exist and are correct dtype
X = X.astype('float32')
y = y.astype('float32')

# Convert to (samples, timesteps, features)
X_seq = X.reshape((-1, 28, 28))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_seq, y, test_size=0.2, random_state=42, shuffle=True)
print("Shapes:", X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# =========================
# Define RNN model
# =========================
def build_rnn_model(time_steps=28, features=28, num_classes=26, rnn_units=128, dropout=0.3):
    inp = layers.Input(shape=(time_steps, features))
    x = layers.Bidirectional(layers.LSTM(rnn_units, return_sequences=True))(inp)
    x = layers.Bidirectional(layers.LSTM(rnn_units // 2))(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(dropout)(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=inp, outputs=out, name="BiLSTM_OCR")
    return model

rnn_model = build_rnn_model()
rnn_model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
rnn_model.summary()

# =========================
# Training setup
# =========================
EPOCHS = 10
BATCH = 128
os.makedirs("models", exist_ok=True)
MODEL_PATH = "models/rnn_ocr_model.keras"

cb = [
    callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    callbacks.ModelCheckpoint(MODEL_PATH, monitor='val_accuracy', save_best_only=True, verbose=1)
]

print("\n🚀 Training started...")
history = rnn_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH,
    callbacks=cb,
    verbose=1
)

# =========================
# Evaluate
# =========================
loss, acc = rnn_model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test accuracy: {acc*100:.2f}% — Loss: {loss:.4f}")

# =========================
# Save a downloadable .h5 model file
# =========================
H5_PATH = "models/rnn_ocr_model.h5"
rnn_model.save(H5_PATH)
print(f"📦 Model saved for frontend: {H5_PATH}")

# =========================
# Plot training curves
# =========================
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.title('Accuracy')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Loss')
plt.legend()
plt.show()

# =========================
# Prediction helper
# =========================
import cv2
def predict_from_image_rnn(image_path, model=rnn_model):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(image_path)
    img = cv2.resize(img, (28,28))
    img = img.astype('float32') / 255.0
    seq = img.reshape(1,28,28)
    pred = model.predict(seq)
    label = np.argmax(pred, axis=1)[0]
    print("Predicted:", chr(label + 65))
    plt.imshow(img, cmap='gray')
    plt.title(f"Predicted: {chr(label+65)}")
    plt.axis('off')
    plt.show()


KeyboardInterrupt: 